In [1]:
import sys
sys.path.append('/home/hice1/awagh31/VLM-Debiasing-Project-1/scripts')

In [2]:
import pandas as pd
import numpy as np

/scratch/956136/ipykernel_1506022/2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:

import os
import model as m
import torch
import torch.nn as nn
import torch.optim as optim
import loaders
from torch.utils.data import Dataset, DataLoader

In [25]:
features_csv = pd.read_csv("/home/hice1/awagh31/scratch/scripts/labels_visual_files_dense.csv")
#features_csv['modalities'] = 'video'

In [26]:
features_csv.head()

,gender,split,PTSD_label,age,PTSD_severity,file_path,video
0,female,train,0,45,22.0,/home/hice1/awagh31/scratch/original/data_unta...,/home/hice1/awagh31/scratch/final_embeddings_v...
1,male,test,0,69,23.0,/home/hice1/awagh31/scratch/original/data_unta...,/home/hice1/awagh31/scratch/final_embeddings_v...
2,male,train,0,25,19.0,/home/hice1/awagh31/scratch/original/data_unta...,/home/hice1/awagh31/scratch/final_embeddings_v...
3,female,train,1,58,67.0,/home/hice1/awagh31/scratch/original/data_unta...,/home/hice1/awagh31/scratch/final_embeddings_v...
4,male,dev,0,33,39.0,/home/hice1/awagh31/scratch/original/data_unta...,/home/hice1/awagh31/scratch/final_embeddings_v...


In [27]:
train_loader = loaders.MultiModalityDataset(features_csv[features_csv["split"]=="train"], 
                                            modalities = {"video"}, label = "PTSD_severity")
dataloader = DataLoader(train_loader, batch_size=16, collate_fn=loaders.collate_fn)

for batch in dataloader:
    modalities, labels = batch
    print(f"Modality 1 shape: {modalities[0].shape}")  # Expected shape: (batch_size, feature, 1)
    #print(f"Modality 2 shape: {modalities[1].shape}")
    #print(f"Modality 3 shape: {modalities[2].shape}")
    print(f"Labels shape: {labels}")
    break

Modality 1 shape: torch.Size([16, 700, 1])
Labels shape: tensor([22., 19., 67., 17., 25., 25., 50., 34., 19., 26., 28., 44., 61., 21.,
        67., 36.])


In [28]:
# Define input dimensions for each modality
input_dims = [768]  # These are the feature dimensions for each modality

# Initialize MultiModalPerceiver model
model = m.MultiModalPerceiver(
    input_dims=input_dims,
    input_channels=1,
    input_axis=1,
    projection_dim=256,
    num_latents=16,
    latent_dim=128,
    depth=8,
    cross_heads=8,
    latent_heads=8,
    cross_dim_head=32,
    latent_dim_head=32,
    attn_dropout=0.1,
    ff_dropout=0.0,
    output_dim=1,
    weight_tie_layers=True,
    fourier_encode_data=False,
    max_freq=10,
    num_freq_bands=4
)

In [29]:
learning_rate = 0.001
num_epochs = 50
# Define loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [31]:
for epoch in range(num_epochs):
    for batch in dataloader:
        # Unpack the batch
        modalities, labels = batch
        modality_1 = modalities[0]  # Each has shape (batch_size, 700, 1)

        # Reshape or project the input to match model requirements
        modality_1 = modality_1.view(modality_1.size(0), -1)  # Flatten to [batch_size, 700]

        # Ensure the input matches the expected size [batch_size, 768]
        if modality_1.size(1) > 768:
            modality_1 = modality_1[:, :768]  # Truncate
        elif modality_1.size(1) < 768:
            padding = torch.zeros((modality_1.size(0), 768 - modality_1.size(1)), dtype=torch.float32).to(modality_1.device)
            modality_1 = torch.cat((modality_1, padding), dim=1)  # Pad

        # Convert modality and labels to float32
        modality_1 = modality_1.float()
        labels = labels.float()

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        inputs = [modality_1]
        output = model(inputs)

        # Reshape labels to match the output shape
        labels = labels.view(output.shape)

        # Compute loss
        loss = criterion(output, labels)
        RMSE_loss = torch.sqrt(loss)

        # Backward pass and optimize
        RMSE_loss.backward()
        optimizer.step()

        # Print loss
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {RMSE_loss.item():.4f}")


Epoch [1/50], Loss: 38.4536
Epoch [1/50], Loss: 34.4295
Epoch [1/50], Loss: 33.8099
Epoch [1/50], Loss: 31.3196
Epoch [1/50], Loss: 37.0714
Epoch [1/50], Loss: 31.9935
Epoch [1/50], Loss: 26.4434
Epoch [1/50], Loss: 40.2472
Epoch [1/50], Loss: 38.6135
Epoch [1/50], Loss: 31.0167
Epoch [1/50], Loss: 15.4004
Epoch [2/50], Loss: 32.2602
Epoch [2/50], Loss: 30.8392
Epoch [2/50], Loss: 31.4963
Epoch [2/50], Loss: 29.6898
Epoch [2/50], Loss: 35.7483
Epoch [2/50], Loss: 30.7799
Epoch [2/50], Loss: 25.2887
Epoch [2/50], Loss: 39.0367
Epoch [2/50], Loss: 37.3092
Epoch [2/50], Loss: 29.7565
Epoch [2/50], Loss: 14.0532
Epoch [3/50], Loss: 31.0144
Epoch [3/50], Loss: 29.5150
Epoch [3/50], Loss: 30.2107
Epoch [3/50], Loss: 28.4880
Epoch [3/50], Loss: 34.5426
Epoch [3/50], Loss: 29.5496
Epoch [3/50], Loss: 24.0889
Epoch [3/50], Loss: 37.7628
Epoch [3/50], Loss: 35.9424
Epoch [3/50], Loss: 28.4573
Epoch [3/50], Loss: 12.6757
Epoch [4/50], Loss: 29.7267
Epoch [4/50], Loss: 28.1241
Epoch [4/50], Loss: 

In [32]:
dev_loader = loaders.MultiModalityDataset(features_csv[features_csv["split"]=="dev"], 
                                            modalities = {"video"}, label = "PTSD_severity")
dev_dataloader = DataLoader(dev_loader, batch_size=16, collate_fn=loaders.collate_fn)

In [34]:
# Model prediction
model.eval()  # Set model to evaluation mode

predictions = []
with torch.no_grad():  # Disable gradient computation
    for batch in dev_dataloader:
        modalities, _ = batch  # Ignore labels if unavailable
        modality_1 = modalities[0]
        
        # Preprocess the input to match model requirements
        modality_1 = modality_1.view(modality_1.size(0), -1)  # Flatten to [batch_size, 700]

        # Ensure the input matches the expected size [batch_size, 768]
        if modality_1.size(1) > 768:
            modality_1 = modality_1[:, :768]  # Truncate
        elif modality_1.size(1) < 768:
            padding = torch.zeros((modality_1.size(0), 768 - modality_1.size(1)), dtype=torch.float32).to(modality_1.device)
            modality_1 = torch.cat((modality_1, padding), dim=1)  # Pad

        # Ensure consistent data type
        modality_1 = modality_1.float()  # Convert to float32

        # Forward pass
        inputs = [modality_1]
        output = model(inputs)  # Model's prediction

        # Collect predictions
        predictions.append(output)

# Combine predictions into a single tensor
predictions = torch.cat(predictions, dim=0)
y_pred = predictions.cpu().numpy()  # Convert to NumPy for further processing
print(f"Predictions:\n{y_pred}")


Predictions:
[[34.10594 ]
 [34.105938]
 [34.105938]
 [34.105938]
 [34.105938]
 [34.105938]
 [34.10594 ]
 [34.10594 ]
 [34.105938]
 [34.10594 ]
 [34.105938]
 [34.105938]
 [34.105938]
 [34.105938]
 [34.10594 ]
 [34.10594 ]
 [34.10594 ]
 [34.105938]
 [34.105938]
 [34.105938]
 [34.105938]
 [34.105938]
 [34.10594 ]
 [34.10594 ]
 [34.105938]
 [34.10594 ]
 [34.105938]
 [34.105938]
 [34.105938]
 [34.105938]
 [34.10594 ]
 [34.10594 ]
 [34.10594 ]
 [34.105938]
 [34.105938]
 [34.105938]
 [34.105938]
 [34.105938]
 [34.10594 ]
 [34.10594 ]
 [34.105938]
 [34.10594 ]
 [34.105938]
 [34.105938]
 [34.105938]
 [34.105938]
 [34.10594 ]
 [34.10594 ]
 [34.105938]
 [34.105938]
 [34.105938]
 [34.10594 ]
 [34.105938]
 [34.105938]
 [34.105938]
 [34.105938]]


In [35]:
y_test = np.array(features_csv[features_csv["split"]=="dev"]["PTSD_severity"].values)

In [36]:
features_csv['col_encoded'] = features_csv['gender'].map({'female': 0, 'male': 1, None:1})

In [37]:
np.unique(features_csv['col_encoded'])

array([ 0.,  1., nan])

In [38]:
gender_train, gender_test = features_csv[features_csv["split"]=="train"]["col_encoded"], features_csv[features_csv["split"]=="dev"]["col_encoded"]

# Ensure that `gender_test`, `y_test`, and `y_pred` are aligned
print(f"Shape of gender_test: {gender_test.shape}")
print(f"Shape of y_test: {y_test.shape}")
print(f"Shape of y_pred: {y_pred.shape}")


Shape of gender_test: (56,)
Shape of y_test: (56,)
Shape of y_pred: (56, 1)


In [45]:
from fairlearn.metrics import demographic_parity_difference
# Calculate Demographic Parity Difference
demographic_parity = demographic_parity_difference(
    y_true=y_test,
    y_pred=y_pred[:,0],
    sensitive_features=gender_test
)

print(f"Demographic Parity Difference: {demographic_parity}")

Demographic Parity Difference: 0.0
